# Assignment: Extracting Static WebPage

Extract information about “วันพระ” for 3 years from:
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx


Note that you can use dateparse package to parse Thai date.  First, we will have to install the package, this is for Google Colab users.  Otherwise, installing via command line is recommended (pip or conda).

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install dateparser

In [2]:
import dateparser

To convert from thai text date string, we will use the parse method.  Note that the parse method assumes the BC year, not BE.  Thus, we will have to subtract 543 from the year.  In addition, weekday() returns day of week with 0=Monday, ..., 6=Sunday.

In [3]:
dt = dateparser.parse('วันศุกร์ที่ 17 มกราคม 2563')

# this will print out weekday == 0 (Monday)
print(dt)
print(dt.weekday())



2563-01-17 00:00:00
0


In [4]:
# this will print out weekday == 4 (Friday)
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2020-01-17 00:00:00
4


In [5]:
dt = dateparser.parse('วันเสาร์ที่ 21 กันยายน 2564')
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2021-09-21 00:00:00
1


In [6]:
from selenium import webdriver
import chromedriver_autoinstaller
import pandas as pd
import os

In [7]:
chromedriver_autoinstaller.install()

'/home/keerati/.conda/envs/dsde-cedt/lib/python3.10/site-packages/chromedriver_autoinstaller/141/chromedriver'

In [8]:
#get by selenium
from selenium.webdriver.common.by import By
browser = webdriver.Chrome()
data_list = []
def web_scraping(url):
    browser.get(url=url)
    table = browser.find_element(By.XPATH, "/html/body/form/div[7]/div[3]/div[10]/div[1]/div/div[2]")
    # if open("raw_data.txt","r"):2
    # with open("raw_data.txt","wa+") as file:
    #         file.append(table)
    #     else:
    #         file.write(table)
    list_text = table.text.split("\n")
    list_text = [text for text in list_text if "ปฏิทินวันพระ" not in text and "วัน" not in text]
    record_list = []
    for idx,text in enumerate(list_text):
        record_list.append(text)
        if (idx+1)%5 == 0:
            data_list.append(record_list)
            record_list = []
year_scrap=[2565,2566,2567]
for year in year_scrap:
    web_scraping(url=f"https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.{year}.aspx")
df = pd.DataFrame(data=data_list, columns=["วัน","วันที่","เดือน","ปี","ข้าง"])
df.to_csv("sum_data.csv")

In [9]:
#get by beautifulSoup


Count the distribution of number of week days that are “วันพระ” for all three years and answer the following questions:

## How many วันพระ in total (of 3 years)?

In [10]:
len(df)

152

## How many days in total (of 3 years) that วันพระ is Monday?

In [11]:
len(df[df["วัน"] == "จันทร์"])

21

## Which day of the week that has the minimum number of วันพระ?

In [12]:
df.groupby(by="วัน").count().sort_values(by="วันที่").index[0]

'อังคาร'

## Which day of the week that has the maximum number of วันพระ?

In [13]:
df.groupby(by="วัน").count().sort_values(by="วันที่", ascending=False).index[0]

'อาทิตย์'